# DNN train MNIST

In [1]:
#import data and library
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images  # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

tf.logging.set_verbosity(old_v)

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
#set outlines 
inputs_num = 784 # 28*28 =784  (784,1)
outputs_num = 10 # 0 to 9
hidden_layer_num = 3000 # 50 nodes per layer                        # 1

#reset variable in tensorflow
tf.reset_default_graph()

#set palceholder to input and output layer
inputs= tf.placeholder(tf.float32,[None,inputs_num],)
targets= tf.placeholder(tf.int32,[None,outputs_num],)

#set first layer variable(xavier) and activation function
w_1 = tf.get_variable('w_1',[inputs_num,hidden_layer_num])
b_1 = tf.get_variable('b_1',[hidden_layer_num])

output_1 = tf.nn.relu(tf.matmul(inputs,w_1)+b_1)                   # 2

#set secod layer variable and activation function

w_2 = tf.get_variable('w_2',[hidden_layer_num,hidden_layer_num])
b_2 = tf.get_variable('b_2',[hidden_layer_num])

output_2 = tf.nn.relu(tf.matmul(output_1,w_2)+b_2)                 # 2

#------------------------------------------------------------------------------------------------------
# add hidden layer
# w_3 = tf.get_variable('w_3',[hidden_layer_num,hidden_layer_num])
# b_3 = tf.get_variable('b_3',[hidden_layer_num])

# output_3 = tf.nn.sigmoid(tf.matmul(output_2,w_3)+b_3)

# w_4 = tf.get_variable('w_4',[hidden_layer_num,hidden_layer_num])
# b_4 = tf.get_variable('b_4',[hidden_layer_num])

# output_4 = tf.nn.relu(tf.matmul(output_3,w_4)+b_4)

# w_5 = tf.get_variable('w_5',[hidden_layer_num,hidden_layer_num])
# b_5 = tf.get_variable('b_5',[hidden_layer_num])

# output_5 = tf.nn.relu(tf.matmul(output_4,w_5)+b_5)

#------------------------------------------------------------------------------------------------------
#set soutput layer variable and activation function

w_3 = tf.get_variable('w_3',[hidden_layer_num,outputs_num])
b_3 = tf.get_variable('b_3',[outputs_num])

outputs = tf.matmul(output_2,w_3)+b_3


#set loss function
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets)   # 3 
loss_mean = tf.reduce_mean(loss)

#set optimization (Adam)
optimize = tf.train.AdamOptimizer(learning_rate=0.00002,).minimize(loss_mean)       # 4

#after softmax, let probability to 0 or 1 by argmax function, and judge between target and output

out_eql_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

#predicte accuracy (need cast to float32)
accuracy = tf.reduce_mean(tf.cast(out_eql_target,tf.float32))

##框架完成


#sess
sess = tf.InteractiveSession()
#Initialization variable
Initilizer = tf.global_variables_initializer()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess.run(Initilizer)

#set SGD batch size
batch_size = 1000                                                                     #5
batch_cycle = mnist.train._num_examples // batch_size

#set early stopping
max_epochs = 15

prev_validation = 9999999.


#training
import time
start = time.time()
log_loss = []
for epoch_counter in range(max_epochs):
    curr_epoc_loss = 0.
    for batch_counter in range(batch_cycle):
        #set input and output batch(iterate by mnist.train.next_batch)
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _,batch_loss = sess.run([optimize,loss_mean], 
                               feed_dict={inputs : input_batch, targets : target_batch})
        curr_epoc_loss += batch_loss
    #read loss per epoch
    curr_epoc_loss /= batch_cycle
    
    #validation every epoch to prevent overfitting()
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    val_loss, val_accuracy = sess.run([loss_mean,accuracy],
                                     feed_dict={inputs:input_batch,targets:target_batch})
    if val_loss>prev_validation:
        break
    prev_validation = val_loss
    
    print(f'Epoch cycle: {epoch_counter + 1}'+
         f' Training loss: {curr_epoc_loss:.3f}'+
         f' Validation loss: {val_loss:.3f}'+
         f' Validation accuracy: {val_accuracy*100.:.2f}'+'%'+
         f' times: {time.time() - start}')
    input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
    test_accuracy = sess.run([accuracy],feed_dict ={inputs:input_batch,targets:target_batch})
    test_accuracy_percent=test_accuracy[0]*100.
    print(f'test accuracy: {test_accuracy_percent:.2f}')
    log_loss.append((epoch_counter+1,curr_epoc_loss,val_loss,val_accuracy,time.time() - start))
print('End of training')
    


E:\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


KeyboardInterrupt: 

In [ ]:
# Tset

input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy],
                         feed_dict ={inputs:input_batch,targets:target_batch})
test_accuracy_percent=test_accuracy[0]*100.
print(f'test accuracy: {test_accuracy_percent:.2f}')

test Hyperparameter
<br />
2 layer
<br />
97.88  1000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=500 layer=5
<br />
<br />
97.41  1000node  sigmoid*2 layer + softmax_cross_entropy  Learning_rate =0.001  batch=500 layer=5
<br />
<br />
97.85  1000node  tahn*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=500 layer=5
<br />
<br />
97.9   1000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=100 layer=5
<br />
<br />
97.99  2000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=100 layer=5
<br />
<br />
97.49  2000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=1000 layer=5
<br />
<br />
97.81  1000node  relu layer      + 2000node relu layer  + softmax_cross_entropy  Learning_rate =0.001  batch=1000 layer=5
<br />
<br />
97.97  1000node  relu layer      + 2000node relu layer  + softmax_cross_entropy  Learning_rate =0.001  batch=1000 layer=5
<br />
<br />
#### 98.05  3000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=1000 layer=5                       # best
<br />
<br />
97.94  3000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.001  batch=100 layer=5
<br />
<br />
98.00 3000node  relu*2 layer    + softmax_cross_entropy  Learning_rate =0.0003  batch=1000 layer=5
<br />
<br />
3 layer
<br />
97.99  1000node  relu*2 layer + sigmoid layer + softmax_cross_entropy  Learning_rate =0.001  batch=100 layer=5 